In [1]:
import numpy as np
import torch
import torchvision
import torchinfo

# Python basic

In [9]:
#Local variable holder.
class TLocalVar:
  def __init__(self, l):
    self.__dict__= l

In [13]:
def test_f(a):
  b= 10
  print(locals())
  l= TLocalVar(locals())
  print(l)
  # print(l['a'])
  print(l.a,l.b)
  l.a*= 100
  l.b*= 100
  print(l.a,l.b)
  print(a,b)
  locals()['a']*= 100
  print(a,b)
test_f(100)

{'b': 10, 'a': 100}
100 10
10000 1000
100 10
100 10


In [14]:
#Container class to share variables.
class TContainer(object):
  def __init__(self):
    pass
  def __del__(self):
    pass
  def __str__(self):
    return str(self.__dict__)
  def __repr__(self):
    return str(self.__dict__)
  def __iter__(self):
    return self.__dict__.itervalues()
  def items(self):
    return self.__dict__.items()
  def iteritems(self):
    return self.__dict__.iteritems()
  def keys(self):
    return self.__dict__.keys()
  def values(self):
    return self.__dict__.values()
  def __getitem__(self,key):
    return self.__dict__[key]
  def __setitem__(self,key,value):
    self.__dict__[key]= value
  def __delitem__(self,key):
    del self.__dict__[key]
  def __contains__(self,key):
    return key in self.__dict__
  def Cleanup(self):
    keys= self.__dict__.keys()
    for k in keys:
      self.__dict__[k]= None
      del self.__dict__[k]

In [15]:
l= TContainer()

In [16]:
for l.i in range(3):
  print(l.i)

0
1
2


# Torch Tensor

## shape and view

In [2]:
# n_batch= 1
n_batch= 4

In [3]:
x= torch.tensor(np.zeros((n_batch,3,32,32))).float()

In [4]:
x.shape

torch.Size([4, 3, 32, 32])

In [5]:
x[0].numel()

3072

In [6]:
x.view(x.size(0), -1).shape

torch.Size([4, 3072])

In [7]:
l=torch.nn.Linear(x[0].numel(), 10)
l

Linear(in_features=3072, out_features=10, bias=True)

In [8]:
l(x[0].view(-1))

tensor([ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
        -0.0003, -0.0002], grad_fn=<AddBackward0>)

In [9]:
l(x.view(x.size(0), -1))

tensor([[ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
         -0.0003, -0.0002],
        [ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
         -0.0003, -0.0002],
        [ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
         -0.0003, -0.0002],
        [ 0.0170, -0.0096, -0.0093, -0.0021,  0.0015,  0.0124,  0.0121,  0.0003,
         -0.0003, -0.0002]], grad_fn=<AddmmBackward>)

## cat (concatenate) and stack

In [10]:
x1= torch.from_numpy(np.array([2,3,4])).float()
x2= torch.from_numpy(np.array([10,22,12])).float()
print(f'x1={x1}')
print(f'x2={x2}')
print(f'torch.cat={torch.cat((x1,x2),axis=0)}')
print(f'torch.stack={torch.stack((x1,x2),axis=0)}')

x1=tensor([2., 3., 4.])
x2=tensor([10., 22., 12.])
torch.cat=tensor([ 2.,  3.,  4., 10., 22., 12.])
torch.stack=tensor([[ 2.,  3.,  4.],
        [10., 22., 12.]])


# view and flatten

In [13]:
n_batch= 64
x= torch.tensor(np.zeros((n_batch,128,6,6))).float()

In [14]:
x.shape

torch.Size([64, 128, 6, 6])

In [19]:
x.view(x.shape[0], -1).shape

torch.Size([64, 4608])

In [23]:
flatten= torch.nn.Flatten()
flatten(x).shape

torch.Size([64, 4608])